# Data

- $C=\{0,1,2,3,4,5,6,7\}$ $\leftarrow$ set of cities to visit.

In [1]:
cities = [c for c in range(8)]

- $D=(d_{ij})$, $i,j \in C$ $\leftarrow$ matrix of distances between cities.

In [2]:
distances = [
    [0, 89, 87, 38, 33, 71, 59, 54],
    [89, 0, 32, 59, 65, 39, 45, 61],
    [87, 32, 0, 50, 75, 17, 64, 79],
    [38, 59, 50, 0, 40, 33, 50, 56],
    [33, 65, 75, 40, 0, 62, 26, 21],
    [71, 39, 17, 33, 62, 0, 57, 70],
    [59, 45, 64, 50, 26, 57, 0, 16],
    [54, 61, 79, 56, 21, 70, 16, 0]
]

# Decision

- $T=\{t_1,t_2,t_3,t_4,t_5,t_6,t_7,t_8,t_9=t_1\}$, $t_i \in C$ $\leftarrow$ ordered set of visited cities.

In [3]:
tour = []

# Objective

The objective is to get a tour $T$ that visits all cities in $C$, ending with the first city, such that the total distance of the tour is as minimal as possible.

$$
\begin{aligned}
& \underset{t \in T}{\text{min}}
& & \sum_{i=1}^{|T|} d_{t_i t_{i+1}}
\end{aligned}
$$

# Procedure

## Initial step

To start the tour, the two nearest cities will be chosen first; that is, we will find the smallest edge in the matrix of distances.

The next function returns the minimum value in a list or set, which is the argument:

In [4]:
def min_notzero(l):
    '''Get minimal value that is greater than zero.'''
    distance = l[1]
    if distance > 0:
        return distance
    else:
        return float('inf')

Now we can get the smallest distance in the matrix, using the function above to exclude the positions of the same cities:

In [5]:
minimums = (min(enumerate(row), key=min_notzero) for _, row in enumerate(distances))
smallest = min(enumerate(minimums), key=lambda m: m[1][1])
smallest[1][1]

16

The smallest edge is $16$, but what we need are the coordinates (the cities) to which that edge belongs:

In [6]:
c1 = smallest[0]
c2 = smallest[1][0]
c1, c2

(6, 7)

Now we can append cities $6$ and $7$ to our tour $T$:

$T=T\oplus\{6, 7\}$

In [7]:
tour.extend([6, 7])
tour

[6, 7]

And remove them from the set of cities $C$:

$\bar{C}=C\setminus\{6,7\}$

In [8]:
cities_bar = cities
cities_bar.remove(6)
cities_bar.remove(7)
cities_bar

[0, 1, 2, 3, 4, 5]

## Nearest Insertion Heuristic

In order to solve the TSP, we will to apply the **nearest insertion heuristic**.

This technique consists of selecting the closest city to the current tour, which is actually a subtour, because we haven't visited all cities yet.

### Nearest function

The closest city is the one whose sum of distances to every visited city in the subtour is the smallest.

$$
\newcommand{\nearest}{\mathop{\mathrm{nearest}}}
\nearest(c \in \bar{C}) = \sum_{t \in T} d_{ct}
$$

In [9]:
def nearest(candidate_city):
    '''Returns the nearest city to the subtour.'''
    return sum(distances[candidate_city][t] for t in tour)

### Cost function

Having the next city to visit already selected, we proceed to choose an edge of the subtour in which the selected city will be inserted. This is determined by calculating the cost of removing the existing edge and creating two new ones, in order to connect the selected city $c_s$, and choose the smallest cost.

$$
\newcommand{\cost}{\mathop{\mathrm{cost}}}
\cost(c_s) = d_{c_st_i} + d_{c_st_{i+1}} - d_{t_it_{i+1}}
$$
$$
t \in T, i \in \{1,\ldots,|T|\}
$$

In [10]:
def cost(selected_city, edge):
    '''Calculates the cost of inserting the selected city in the edge.'''
    t0, t1 = edge
    distance_cs_t0 = distances[selected_city][t0]
    distance_cs_t1 = distances[selected_city][t1]
    distance_t0_t1 = distances[t0][t1]
    return distance_cs_t0 + distance_cs_t1 - distance_t0_t1